In [6]:
# Welcome to your new notebook
# Type here in the cell editor to add code!

# Check what database/catalog we are currently using
print("current_catalog:", spark.sql("SELECT current_catalog()").collect()[0][0])
print("current_database:", spark.sql("SELECT current_database()").collect()[0][0])




StatementMeta(, 9f061f19-0290-4679-b6e6-96ed5e28eef3, 8, Finished, Available, Finished)

current_catalog: spark_catalog
current_database: chimcobldhq2ajbp41rmusjbedo62or54lo6usjkcpnmoqbfbto74qbdc5p7inr3c5p6anrcc5lmaq3felpma9b4c9ng


In [9]:
#after error saying
#[SCHEMA_NOT_FOUND] The schema `default.My workspace.portfolio_primary_care_lakehouse.silver` cannot be found. 
#Verify the spelling and correctness of the schema and catalog.
spark.sql("CREATE SCHEMA IF NOT EXISTS silver")


StatementMeta(, 9f061f19-0290-4679-b6e6-96ed5e28eef3, 11, Finished, Available, Finished)

DataFrame[]

In [47]:

from pyspark.sql.functions import col, to_date, when

## read from bronze to be updated with silver rules
df_bronze = spark.table("bronze.appointments_raw")

df_silver = (
    df_bronze
    .withColumn("Age", col("Age").cast("int"))
    .withColumn("appointment_date", to_date("appointment_date", "dd/MM/yyyy"))
    .withColumn(
        "Gender",
        when(col("Gender").isin("Male","Female"), col("gender"))
        .otherwise("Unknown")
    )
.filter(col("practice_code").isNotNull())
.filter(col("appointment_date").isNotNull())

)

df_silver.write.mode("overwrite").saveAsTable("silver.appointments_clean")

StatementMeta(, 9f061f19-0290-4679-b6e6-96ed5e28eef3, 49, Finished, Available, Finished)

In [31]:
# check i data types are correct
df_bronze.printSchema()


StatementMeta(, 9f061f19-0290-4679-b6e6-96ed5e28eef3, 33, Finished, Available, Finished)

root
 |-- emis_number: string (nullable = true)
 |-- usual_gp_org_name: string (nullable = true)
 |-- practice_code: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- appointment_date: date (nullable = true)
 |-- appointment_time: string (nullable = true)
 |-- slot_type: string (nullable = true)
 |-- current_slot_status: string (nullable = true)
 |-- location_name: string (nullable = true)
 |-- session_holder: string (nullable = true)



In [48]:
#df_silver.head()
spark.table("silver.appointments_clean").count()
#spark.table("bronze.appointments_raw").head()

#df_bronze.head()

StatementMeta(, 9f061f19-0290-4679-b6e6-96ed5e28eef3, 50, Finished, Available, Finished)

10